In [98]:
from riotwatcher import LolWatcher, ApiError
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from datetime import datetime
import json

# current date
now = datetime.now()

date = now.strftime("%d %b, %Y")

lw = LolWatcher('RGAPI-0bb964a5-de77-4126-93cb-dbb50ea64a0f')

summ_list = [
    'GrandfatherFlesh','satorian','Superpen1sball',
    '36CYOB','Rumahoy','Tobias Fate','SoloRenektonOnly',
    'shaanathan','be pure','Mistress Lolly','Falcon1124',
    'TFBlade2','Isles','Tempos Time','The Tank Man']

my_region = 'americas'
filePath = "/Users/danielmoran/Documents/School/ML/MLProject_2021/Dataset/" + date
print(filePath)
Games=[]

for summ in summ_list:
    me = lw.summoner.by_name('na1', summ)
    
    offset = 0
    #0 - 12/5
    #1000 - 12/6
    #2000 - 12/7
    print(summ)
    my_matches = lw.match.matchlist_by_puuid('americas', me['puuid'], start = offset, type="normal", count = 100)
    my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], start = offset, type="ranked", count = 100)
    for i in range (1,10):
        my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], type="normal", start = (i*100+offset), count = 100)
        my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], type="ranked", start = (i*100+offset), count = 100)
        
    n_games = 1000
    j=0
    count=0
    while count < n_games:
        try:
            time.sleep(1)
            last_match = my_matches[count]
            count+=1
            #print(last_match)
            game_info = lw.match.by_id(my_region, last_match)
            winner = game_info['info']['teams'][0]['win']
            #print(winner)
            #print("goldP1 : \n", lw.match.timeline_by_match(my_region, last_match)['info']['frames'][-1]['participantFrames']['1']['totalGold'])
            data = lw.match.timeline_by_match(my_region, last_match)['info']['frames']
            for event in data:
                #print(type(event),'\n',type(event.items()),event.items())
                if "participantFrames" in event:
                    #print('\n\n\n',"EVENT : ",type(event),'\n',event)
                    #print('\n\n\n',"TS : ",event['timestamp'])
                    if event['timestamp'] <= 900000:
                        selection = event
                    if event['timestamp'] > 900000:
                        if abs(selection['timestamp']-900000) > abs(event['timestamp']-900000):
                            selection = event
                            break
                        else:
                            break
            event_data = [selection['participantFrames'], selection['timestamp']]
            #print(json.dumps(event_data, indent=2))

            frame = event_data[0]
            game = np.zeros(31)
            i = 0

            for player in range(1, 11):
                game[i] = frame[str(player)]['damageStats']['totalDamageDoneToChampions']
                i+=1
                game[i] = frame[str(player)]['totalGold']
                i+=1
                game[i] = frame[str(player)]['xp']
                i+=1
            if winner == True:
                game[i] = 1
            else:
                game[i] = 0

            #print(game)
            Games.append(game)
        except:
            count+=1

dataset = pd.DataFrame(Games, columns = [
                        'Player_1_Damage','Player_1_Gold','Player_1_XP',
                        'Player_2_Damage','Player_2_Gold', 'Player_2_XP',
                        'Player_3_Damage','Player_3_Gold', 'Player_3_XP',
                        'Player_4_Damage','Player_4_Gold', 'Player_4_XP',
                        'Player_5_Damage','Player_5_Gold', 'Player_5_XP',
                        'Player_6_Damage','Player_6_Gold', 'Player_6_XP',
                        'Player_7_Damage','Player_7_Gold', 'Player_7_XP',
                        'Player_8_Damage','Player_8_Gold', 'Player_8_XP',
                        'Player_9_Damage','Player_9_Gold', 'Player_9_XP',
                        'Player_10_Damage','Player_10_Gold', 'Player_10_XP',
                        'Victory'])

dataset.head()

dataset.to_csv(filePath)

/Users/danielmoran/Documents/School/ML/MLProject_2021/Dataset/05 Dec, 2021


HTTPError: 403 Client Error: Forbidden for url: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/GrandfatherFlesh

In [97]:
dataset.head()

(2884, 31)